In [1]:
import math
import ast
import statistics
import numpy as np
import os
import csv
import json
from collections import defaultdict
import scipy.stats as stats
import matplotlib.pyplot as plt
import random as nd
import pickle as pkl
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
%pylab inline
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


# Main data parsing functions

In [2]:
def save_plot_data(obj, name, path):
    with open(path+name+'.pkl', 'wb') as f:
        pkl.dump(obj, f, pkl.HIGHEST_PROTOCOL)

def load_plot_data(name, path):
    with open(path+name+'.pkl', 'rb') as f:
        return pkl.load(f)

def load_json_data(inputfile):
    summ_data = []
    
    with open(inputfile) as json_file:
        summ_data = json.load(json_file)
        print('agent_ID: ' + summ_data['agent_ID'])
        #print('agent_type: ' + summ_data['agent_type'])
        #print('total_episodes: ' + str(summ_data['total_episodes']))
        #print('total_reward: ' + str(summ_data['total_reward']))
        #print('avg_rec_err: ' + str(summ_data['avg_rec_err']))
        #print('CL_ratio: ' + str(summ_data['CL_ratio']))
        #print('STM_length: ' + str(summ_data['STM_length']))
        #print('rec_thres: ' + str(summ_data['rec_thres']))
        #print('mean_step_rew: ' + str(summ_data['mean_step_rew']))
        #print('mean_step_log: ' + data['mean_step_log'])
        #print('wins_reactive: ' + str(summ_data['wins_reactive']))
        #print('wins_contextual: ' + str(summ_data['wins_contextual']))
        #print('auto_reliable: ' + str(summ_data['auto_reliable']))

    return summ_data

def load_csv_data(inputfile):

    # initializing the titles and rows list 
    fields = [] 
    data = [] 

    # reading csv file 
    with open(inputfile, 'r') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 

        # extracting field names through first row  
        fields = next(csvreader) 
        
        # printing the field names 
        #print('Field names are: ' + ', '.join(field for field in fields)) 
        
        # extracting each data row one by one 
        for row in csvreader: 
            if len(row) > 0:
                data.append(row) 
               
        #print('Data length: ', len(data))

        # get total number of rows 
        print("Total no. of rows: %d"%(csvreader.line_num)) 
    
    print("Data length: ", len(data))  

    return data

def get_reward_ep(data):
    reward_ep = []
    for d in data:
        reward_ep.append(d[0])
    return reward_ep

def get_rec_error_mean(data):
    rec_error_mean = []
    for d in data:
        rec_error_mean.append(d[1])
    return rec_error_mean

def get_reward_logs(data):
    reward_logs = []
    for d in data:
        reward_logs.append(d[2])
    return reward_logs

def get_rec_error_logs(data):
    rec_error_logs = []
    for d in data:
        rec_error_logs.append(d[3])
    return rec_error_logs

def get_agent_speed_logs(data):
    agent_speed_logs = []
    for d in data:
        agent_speed_logs.append(d[4])
    return agent_speed_logs

def get_agent_pos_logs(data):
    agent_pos_logs = []
    for d in data:
        agent_pos_logs.append(d[5])
    return agent_pos_logs

def get_active_layer_logs(data, model):
    active_layer_logs = []
    for d in data:
        active_layer_logs.append(d[6])
        #active_layer_logs.append(d[6]) if model != 'MFEC' else active_layer_logs.append(d[5])
    return active_layer_logs

def get_entropy_logs(data, model):
    entropy_logs = []
    for d in data:
        entropy_logs.append(d[6])
        #if model != 'MFEC': entropy_logs.append(d[7]) 
    return entropy_logs

def get_action_sel_logs(data, model):
    action_sel_logs = []
    for d in data:
        action_sel_logs.append(d[8])
        #action_sel_logs.append(d[8]) if model != 'MFEC' else action_sel_logs.append(d[6])
    return action_sel_logs

def get_reward_ts(reward_ep, ltm_length):
    #ltm = int(ltm_length[3:])
    ltm = ltm_length

    abs_tru_reward_ts = []
    abs_rnd_reward_ts = []
    acc_tru_reward = 0
    acc_tru_reward_ts = []
    acc_rnd_reward = 0
    acc_rnd_reward_ts = []
    count_ltm = 0
    index_eps = 0
    ltm_complete = 0
    
    for episode in reward_ep:
        #print(episode)
        r = float(episode)
        
        #print(r)
        if r > 0:
            #print(r)
            abs_tru_reward_ts.append(r)
            abs_rnd_reward_ts.append(3)
            acc_tru_reward += r
            acc_rnd_reward += 3
            count_ltm += 1
        else:
            abs_tru_reward_ts.append(0)
            abs_rnd_reward_ts.append(0)
            acc_tru_reward += 0
            acc_rnd_reward += 0
            
        if count_ltm == ltm: 
            ltm_complete = index_eps
            ltm = -1
            
        acc_tru_reward_ts.append(acc_tru_reward)
        acc_rnd_reward_ts.append(acc_rnd_reward)
        index_eps += 1
        
    return abs_tru_reward_ts, abs_rnd_reward_ts, acc_tru_reward_ts, acc_rnd_reward_ts, ltm_complete


def get_steps_ts(reward_ep, steps_data, cl=4):
    #default episode total actions: 
    # CL4 - 251, CL2 - 501, # CL1 - 1001
    #print(cl[1])
    #default_steps = round(1000 / int(cl[1:]))
    default_steps = round(1000 / cl)
    steps_index = 0
    ep_steps_ts = []
    acc_steps = 0
    acc_steps_ts = []
    steps = steps_data
    #steps = json_data['mean_step_log']
    #for episode in reward_ep:
    for episode in reward_ep:
        #print(episode)
        r = float(episode)
        #print(r)
        if r > 0:
            ep_steps_ts.append(int(steps[steps_index]))
            acc_steps += int(steps[steps_index])
            steps_index += 1 
        else:
            ep_steps_ts.append(default_steps)
            acc_steps += default_steps
            
        acc_steps_ts.append(acc_steps)    
                    
    return ep_steps_ts, acc_steps_ts

def get_condition_data(cl, stm):
    data = []
    for r,d,f in os.walk(base_path):
        for files in f:
            if files.startswith('c'+str(cl)+'-stm'+str(stm)) and files.endswith('.csv'):
                #ts = []
                inputfile = r + files
                print("r: ", r)
                data = load_data(inputfile)
    return data


def get_layer_reliance_ep(layer_logs):
    reactive_reliance = []
    contextual_reliance = []
    episode_dominance = []
    episode_last = []
    for ep in layer_logs:
        contextual = 0
        reactive = 0
        for d in ep:
            if d == 'C': contextual += 1
            if d == 'R': reactive += 1
        if contextual >= reactive: episode_dominance.append('C')
        else: episode_dominance.append('R')
        if (reactive+contextual) > 0:
            rel_reactive = reactive / (reactive+contextual)
            rel_contextual = contextual / (reactive+contextual)
        else:
            rel_reactive = 0
            rel_contextual = 0
        reactive_reliance.append(rel_reactive)
        contextual_reliance.append(rel_contextual)
        episode_last.append(ep[-3])
    return episode_dominance, episode_last, reactive_reliance, contextual_reliance


def get_layer_reward_ts_last(reward_ep, steps_data, ep_last):
    count_reactive = 0
    abs_reward_reactive = []
    abs_reward_contextual = []
    acc_contextual_reward = 0
    acc_reactive_reward = 0
    acc_reward_reactive = []
    acc_reward_contextual = []
    #count_steps = 0
    steps_reactive = []
    steps_contextual = []
    steps = steps_data
    #steps = json_data['mean_step_log']
    #for episode in reward_ep:
    for i in range (len(reward_ep)):
        #print(episode)
        r = float(reward_ep[i])
        #print(r)
        if r >= 1.:
            #if count_reactive > 4:
            if ep_last[i] == 'R':
                acc_reactive_reward += r
                abs_reward_reactive.append(r)
                acc_reward_reactive.append(acc_reactive_reward)
                steps_reactive.append(steps[count_reactive])
                
                abs_reward_contextual.append(0)
                acc_reward_contextual.append(acc_contextual_reward)
            else:
                acc_contextual_reward += r
                abs_reward_contextual.append(r)
                acc_reward_contextual.append(acc_contextual_reward)
                steps_contextual.append(steps[count_reactive])  
                
                abs_reward_reactive.append(0)
                acc_reward_reactive.append(acc_reactive_reward)
                    
            count_reactive += 1                
            #count_steps += 1  
        
    return abs_reward_reactive, acc_reward_reactive, abs_reward_contextual, acc_reward_contextual, steps_reactive, steps_contextual

def get_mean_entropy(data):
    mean_entropy_ep = []
    for d in data:
        #a = np.fromstring(d[1:-1], dtype=float, sep=',')
        float_array = ast.literal_eval(d)
        mean_entropy = np.mean(float_array)
        mean_entropy_ep.append(mean_entropy)
    return mean_entropy_ep

def get_encoder_reliability_ts(rec_error_logs, rec_thresh=0):
    data = rec_error_logs
    reconstruct_thres = rec_thresh
    encoder_reliability_ts = []

    for ep in data:
        a = np.fromstring(ep[1:-1], dtype=float, sep=',')
        #print(len(a))
        count_reliable = 0
        count_unreliable = 0
        encoder_reliability = 0
        for rec_error_ts in a:
            if rec_error_ts < reconstruct_thres: 
                #print ('Reconstruction Error RELIABLE')
                count_reliable +=1  
            else:
                #print ('Reconstruction Error UNRELIABLE') 
                count_unreliable +=1
        encoder_reliability = (count_reliable / (count_reliable+count_unreliable)) * 100
        encoder_reliability_ts.append(encoder_reliability)
    return encoder_reliability_ts

In [3]:
def get_condition_dac(filepath, model, ltm_length=500):
    
    agent = "dac"
    indx = 0
    jsn_data = 0
    agent_ID = ""
    
    rec_threshold=0.01

    #big_list = []
    big_json = []
    big_steps = []
    big_autoencoder_trust_rel, big_autoencoder_trust_ts = [], []
    big_reward_ep = []
    big_abs_tru_reward, big_abs_rnd_reward, big_acc_tru_reward, big_acc_rnd_reward = [], [], [], []
    big_layer_data = []
    big_ep_dominance, big_ep_last, big_reactive_rel, big_contextual_rel =  [], [], [], []
    big_abs_rew_reactive, big_acc_rew_reactive, big_abs_rew_contextual, big_acc_rew_contextual, big_steps_reactive, big_steps_contextual = [], [], [], [], [], []
    big_rec_error_logs = []
    big_wins_reactive, big_wins_contextual = [], []
    #big_autoencoder_trust_ts = []  
    big_entropy_data, big_mean_entropy = [], []
    big_real_steps, big_acc_steps = [], []
    big_ltm_complete = []
    big_pos = []

    print(filepath+'{0}/'.format(model))
    for r,d,f in os.walk(filepath+'{0}/'.format(model)):
        #print("oink! 0")
        for files in f:
            if files.endswith('.json'):
                filename = r+'/'+files
                jsn_data = load_json_data(filename)
                agent_ID = jsn_data['agent_ID']
                agent_type = jsn_data['agent_type']
                big_json.append(jsn_data)
                big_autoencoder_trust_rel.append(jsn_data['auto_reliable'])
                mean_steps_log = jsn_data['mean_step_log']
                big_steps.append(mean_steps_log)
                big_wins_reactive.append(jsn_data['wins_reactive'])
                big_wins_contextual.append(jsn_data['wins_contextual'])
                #print("oink! 1: ", agent_ID)
                for r,d,f in os.walk(filepath+'{0}/'.format(model)):
                    #print("oink! 2a")
                    for files in f:
                        if files.startswith(agent_ID) and files.endswith('.csv'):
                            #print("oink! 2b: ", agent_ID)
                            filename = r+'/'+files
                            #print("oink! 2c: ", filename)
                            csv_data = load_csv_data(filename)
                            reward_ep = get_reward_ep(csv_data)
                            big_reward_ep.append(reward_ep)
                            abs_tru_reward, abs_rnd_reward, acc_tru_reward, acc_rnd_reward, ltm_full = get_reward_ts(reward_ep, ltm_length)
                            big_abs_tru_reward.append(abs_tru_reward)
                            big_abs_rnd_reward.append(abs_rnd_reward)
                            big_acc_tru_reward.append(acc_tru_reward)
                            big_acc_rnd_reward.append(acc_rnd_reward)
                            big_ltm_complete.append(ltm_full)
                            if agent_type != "reactive": 
                                rec_error_logs = get_rec_error_logs(csv_data)
                                big_rec_error_logs.append(rec_error_logs)
                                #autoencoder_trust_ts = get_encoder_reliability_ts(rec_error_logs, rec_threshold)
                                #big_autoencoder_trust_ts.apppend(autoencoder_trust_ts)
                                entropy_data = get_entropy_logs(csv_data, agent_type)
                                mean_entropy = get_mean_entropy(entropy_data)
                                big_entropy_data.append(entropy_data)
                                big_mean_entropy.append(mean_entropy)
                            #if model == "dac-pos" or model == "dac-nobias-pos":
                                #agent_pos_logs = get_agent_pos_logs(csv_data)
                                #big_pos.append(agent_pos_logs)
                            indx += 1
                      
    big_list = {}

    #big_list['agent_ID'] = ID
    big_list['abs_tru_reward'] = big_abs_tru_reward
    big_list['acc_tru_reward'] = big_acc_tru_reward  #this
    big_list['reward_steps_ts'] = big_steps
    big_list['real_steps_ts'] = big_real_steps  #this
    big_list['acc_steps_ts'] = big_acc_steps  #this
    big_list['mean_entropy'] = big_mean_entropy #this
    big_list['LTM_full'] = big_ltm_complete 
                
    return big_list

# Load data

In [165]:
file_path = '../data/exp1_2tmaze/'
plot_path = '../data/exp1_2tmaze/'

In [166]:
#data = load_plot_data('data', file_path)
data = load_plot_data('data', file_path)

In [167]:
data.keys()

dict_keys(['MFEC-ae', 'MFEC-rp', 'SEC', 'NSEC', 'DQN', 'ERLAM'])

In [168]:
data['SEC'].keys()

dict_keys(['abs_tru_reward', 'acc_tru_reward', 'reward_steps_ts', 'real_steps_ts', 'acc_steps_ts', 'mean_entropy', 'LTM_full', 'reactive_relact', 'contextual_relact', 'reactive_wins', 'contextual_wins', 'abs_rew_reactive', 'abs_rew_contextual', 'acc_rew_reactive', 'acc_rew_contextual', 'steps_reactive', 'steps_contextual'])

# Rewards - all

In [169]:
#models = ['SEC_500', 'SEC_1000', 'MFEC_VAE_25K', 'MFEC_RP_25K', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'DQN_5k', 'DQN_25k', 'ERLAM_5k', 'ERLAM_25k']
#models = ['noGi_500', 'NSEC_500', 'NSEC_1000', 'SEC_500', 'SEC_1000', 'MFEC_VAE_25K', 'MFEC_RP_25K', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'DQN_5k', 'DQN_25k', 'ERLAM_5k', 'ERLAM_25k']
#models = ['NSEC_500', 'SEC_500', 'SEC_1000', 'MFEC_VAE_25K', 'MFEC_RP_25K', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'DQN_5k', 'DQN_25k', 'ERLAM_5k', 'ERLAM_25k']
#models = ['noGi_500', 'NSEC_500', 'SEC_500', 'SEC_1000', 'MFEC_VAE_25K', 'MFEC_RP_25K', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'ERLAM_5k', 'ERLAM_25k', 'DQN_5k', 'DQN_25k']
#models = ['DQN_ae100_cylinder', 'DQN_sparse100_cylinder', 'DQN_ae1k_cylinder', 'DQN_sparse1k_cylinder', 'MFEC_ae100_cylinder', 'MFEC_sparse100_cylinder', 'SEC_ae100_cylinder', 'SEC_sparse100_cylinder']
#models = ['DQN_ae100_cylinder', 'DQN_sparse100_cylinder', 'MFEC_ae100_cylinder', 'MFEC_sparse100_cylinder', 'SEC_ae100_cylinder', 'SEC_sparse100_cylinder']
#models = ['noGi_500', 'NSEC_500', 'NSEC_1000', 'SEC_500', 'SEC_1000', 'MFEC_VAE_25K', 'MFEC_RP_25K', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'ERLAM_5k', 'ERLAM_25k', 'DQN_5k', 'DQN_25k']
#models = ['DQN_ae100_'+task, 'DQN_sparse100_'+task, 'MFEC_ae100_'+task, 'MFEC_sparse100_'+task, 'SEC_ae100_'+task, 'SEC_sparse100_'+task]
#models = ['DQN_ae1k_'+task, 'DQN_sparse1k_'+task]
data = load_plot_data('data', file_path)
models = ['MFEC-ae', 'MFEC-rp', 'SEC', 'NSEC', 'DQN', 'ERLAM']
ltm = 500

In [170]:
import numpy as np
from scipy.stats import mannwhitneyu
import itertools

In [171]:
data_trureward = data['SEC']['abs_tru_reward']
data_trureward = np.asarray(data_trureward, dtype=np.float32)
print("data_truerward_sum ", np.sum(data_trureward))
print("data_truerward shape ", np.shape(data_trureward))

if np.sum(data_trureward) > 0:
    if len(data_trureward[0]) > 5000: 
        data_half = np.hsplit(data_trureward, 2)
        data_trureward = data_half[0]

    mean = np.mean(data_trureward, axis=0)
    print("last mean 1", mean[-1])
    last_elements = data_trureward[:, -1]
    last_mean = np.mean(last_elements)
    print("last mean 2", last_mean)
    print(len(last_elements))

data_truerward_sum  232097.61
data_truerward shape  (20, 5000)
last mean 1 2.78515
last mean 2 2.78515
20


In [174]:
task = 'DTmaze'
models = ['MFEC-ae', 'MFEC-rp', 'SEC', 'NSEC', 'DQN', 'ERLAM']

reward_data = {}
window_width = 20

for model in models:

    data_trureward = data[model]['abs_tru_reward']
    data_trureward = np.asarray(data_trureward, dtype=np.float32)
    print("data_truerward_sum ", np.sum(data_trureward))

    if np.sum(data_trureward) > 0:
        if len(data_trureward[0]) > 5000: 
            data_half = np.hsplit(data_trureward, 2)
            data_trureward = data_half[0]

        mean = np.mean(data_trureward, axis=0)
        std = np.std(data_trureward, axis=0)
        
        last_elements = data_trureward[:, -1]
        #last_mean = np.mean[-1]
        last_mean = np.mean(last_elements)
        #last_std = std[-1]
        last_std = np.std(last_elements)
        #print("reward mean length ", len(mean))
        print("reward mean last ", last_mean)
        print("reward std last ", last_std)

        yerr = np.std(data_trureward, axis=0) / math.sqrt(len(data_trureward))
        last_yerr = np.std(last_elements) / math.sqrt(len(last_elements))
        print("reward yerr last", yerr[-1])
        print("reward yerr last 2 ", last_yerr)

        data_trureward_cumsum = np.cumsum(np.insert(mean, 0, 0)) 
        data_trureward_mavecr = (data_trureward_cumsum[window_width:] - data_trureward_cumsum[:-window_width]) / window_width

        data_trureward_cumsum_std = np.cumsum(np.insert(std, 0, 0)) 
        data_trureward_mavecr_std = (data_trureward_cumsum_std[window_width:] - data_trureward_cumsum_std[:-window_width]) / window_width

        data_trureward_cumsum_yerr = np.cumsum(np.insert(yerr, 0, 0)) 
        data_trureward_mavecr_yerr = (data_trureward_cumsum_yerr[window_width:] - data_trureward_cumsum_yerr[:-window_width]) / window_width

        data_totalreward = np.sum(data_trureward, axis=1) 
        print("total reward length ", len(data_totalreward))
        print("total reward ", data_totalreward)
        data_totalreward_mean = np.mean(data_totalreward)
        data_totalreward_std = np.std(data_totalreward)
        data_totalreward_yerr = data_totalreward_std / math.sqrt(len(data_totalreward))

        reward = {}
        reward['total'] = data_totalreward
        reward['total_mean'] = data_totalreward_mean
        reward['total_std'] = data_totalreward_std
        reward['total_yerr'] = data_totalreward_yerr
        reward['rwds_last'] = last_elements
        reward['mean_hist'] = mean
        reward['mean_last'] = last_mean
        reward['std_hist'] = std
        reward['std_last'] = last_std
        reward['yerr_hist'] = yerr
        reward['yerr_last'] = last_yerr
        reward['mean'] = np.mean(mean)
        reward['mavecr'] = data_trureward_mavecr
        reward['mavecr_std'] = data_trureward_cumsum_std
        reward['mavecr_yerr'] = data_trureward_mavecr_yerr
        reward['min_val'] = data_trureward_mavecr - data_trureward_mavecr_yerr
        reward['max_val'] = data_trureward_mavecr + data_trureward_mavecr_yerr
        reward['length'] = data_trureward_mavecr_std.shape[0]

        reward_data[model] = reward

save_plot_data(reward_data, 'reward_new_'+task, file_path)

data_truerward_sum  70509.586
reward mean last  1.0884
reward std last  1.33557
reward yerr last 0.29864252
reward yerr last 2  0.2986425258979946
total reward length  20
total reward  [2391.732    593.8881    10.793   9521.001   8964.163   5782.039
 4974.2354  2598.361    431.44202  208.48201   11.875   4494.66
  540.274   5921.997   3785.2139  5425.5176  4627.433   1876.476
 2597.143   5752.8604 ]
data_truerward_sum  22324.096
reward mean last  0.717
reward std last  1.2418815
reward yerr last 0.35850027
reward yerr last 2  0.3585003062054628
total reward length  12
total reward  [8222.271    7534.708      10.856001   11.925001   11.360001   11.831001
  821.27704    12.305      44.953003  406.077      11.666001 5224.865   ]
data_truerward_sum  232097.61
reward mean last  2.78515
reward std last  0.124701716
reward yerr last 0.027884152
reward yerr last 2  0.027884151389984718
total reward length  20
total reward  [12761.725  11985.011  11172.252  12745.375  12343.921  10913.979
 1197

In [5]:
#data = load_plot_data('data', file_path)
file_path = '../data/exp4_forgetting/'
plot_path = '../data/exp4_forgetting/'
data = load_plot_data('data_new', file_path)
data.keys()

dict_keys(['MFEC', 'SEC', 'SEC-FIFO', 'SEC-RWD', 'NSEC', 'NSEC-FIFO', 'NSEC-RWD', 'SEC-FIFO_v2', 'SEC-RWD_v2', 'SEC-RWD_v3', 'SEC-RWD_v4', 'SEC-PRIOR'])

In [6]:
task = 'Forgetting'
models = ['SEC', 'SEC-PRIOR', 'SEC-FIFO_v2', 'NSEC', 'NSEC-FIFO', 'NSEC-RWD']

reward_data = {}
window_width = 20

for model in models:

    data_trureward = data[model]['abs_tru_reward']
    data_trureward = np.asarray(data_trureward, dtype=np.float32)
    print("MODEL: ", model)
    print("data_truerward_sum ", np.sum(data_trureward))

    if np.sum(data_trureward) > 0:
        if len(data_trureward[0]) > 5000: 
            data_half = np.hsplit(data_trureward, 2)
            data_trureward = data_half[0]

        mean = np.mean(data_trureward, axis=0)
        std = np.std(data_trureward, axis=0)
        
        last_elements = data_trureward[:, -1]
        #last_mean = np.mean[-1]
        last_mean = np.mean(last_elements)
        #last_std = std[-1]
        last_std = np.std(last_elements)
        #print("reward mean length ", len(mean))
        print("reward mean last ", last_mean)
        print("reward std last ", last_std)

        yerr = np.std(data_trureward, axis=0) / math.sqrt(len(data_trureward))
        last_yerr = np.std(last_elements) / math.sqrt(len(last_elements))
        print("reward yerr last", yerr[-1])
        print("reward yerr last 2 ", last_yerr)

        data_trureward_cumsum = np.cumsum(np.insert(mean, 0, 0)) 
        data_trureward_mavecr = (data_trureward_cumsum[window_width:] - data_trureward_cumsum[:-window_width]) / window_width

        data_trureward_cumsum_std = np.cumsum(np.insert(std, 0, 0)) 
        data_trureward_mavecr_std = (data_trureward_cumsum_std[window_width:] - data_trureward_cumsum_std[:-window_width]) / window_width

        data_trureward_cumsum_yerr = np.cumsum(np.insert(yerr, 0, 0)) 
        data_trureward_mavecr_yerr = (data_trureward_cumsum_yerr[window_width:] - data_trureward_cumsum_yerr[:-window_width]) / window_width

        data_totalreward = np.sum(data_trureward, axis=1) 
        print("total reward length ", len(data_totalreward))
        print("total reward ", data_totalreward)
        data_totalreward_mean = np.mean(data_totalreward)
        data_totalreward_std = np.std(data_totalreward)
        data_totalreward_yerr = data_totalreward_std / math.sqrt(len(data_totalreward))

        reward = {}
        reward['total'] = data_totalreward
        reward['total_mean'] = data_totalreward_mean
        reward['total_std'] = data_totalreward_std
        reward['total_yerr'] = data_totalreward_yerr
        reward['rwds_last'] = last_elements
        reward['mean_hist'] = mean
        reward['mean_last'] = last_mean
        reward['std_hist'] = std
        reward['std_last'] = last_std
        reward['yerr_hist'] = yerr
        reward['yerr_last'] = last_yerr
        reward['mean'] = np.mean(mean)
        reward['mavecr'] = data_trureward_mavecr
        reward['mavecr_std'] = data_trureward_cumsum_std
        reward['mavecr_yerr'] = data_trureward_mavecr_yerr
        reward['min_val'] = data_trureward_mavecr - data_trureward_mavecr_yerr
        reward['max_val'] = data_trureward_mavecr + data_trureward_mavecr_yerr
        reward['length'] = data_trureward_mavecr_std.shape[0]

        reward_data[model] = reward

save_plot_data(reward_data, 'reward_new_'+task, file_path)

MODEL:  SEC
data_truerward_sum  232097.61
reward mean last  2.78515
reward std last  0.124701716
reward yerr last 0.027884152
reward yerr last 2  0.027884151389984718
total reward length  20
total reward  [12761.725  11985.011  11172.252  12745.375  12343.921  10913.979
 11974.027  11929.746  12451.729   6251.8413 12884.659  11486.684
 11635.516  12759.763  12604.989  12912.76   10759.244  11210.25
 13151.1875  8162.9517]
MODEL:  SEC-PRIOR
data_truerward_sum  243838.2
reward mean last  2.6044497
reward std last  0.6151484
reward yerr last 0.13755135
reward yerr last 2  0.1375513694780533
total reward length  20
total reward  [13000.182  12393.184  13217.7705 11684.592  13195.868  12461.193
 12658.344  13015.821  12118.3    12056.24   11674.811  11980.355
 11859.637  10715.049  12475.845  12769.166  12519.915  12344.092
 12472.881   9224.965 ]
MODEL:  SEC-FIFO_v2
data_truerward_sum  248028.42
reward mean last  2.7922003
reward std last  0.14212337
reward yerr last 0.03177975
reward yerr

## Statistics v1

In [57]:
import numpy as np
import math
import pandas as pd
from scipy.stats import mannwhitneyu
import itertools
import matplotlib.pyplot as plt

def extract_final_rewards(data, models, end_window_size=100):
    """
    Extract final rewards from the last `end_window_size` episodes for each model.
    
    Parameters:
    - data: Dictionary containing reward data for each model.
    - models: List of model names to extract.
    - end_window_size: Number of episodes to consider from the end.
    
    Returns:
    - final_rewards: Dictionary with final rewards for each model.
    """
    final_rewards = {model: data[model]['abs_tru_reward'][:, -end_window_size:].mean(axis=1) for model in models}
    return final_rewards

def perform_statistical_tests(final_rewards, models, base_model='SEC'):
    """
    Perform pairwise Mann-Whitney U tests comparing the base_model against other models.
    
    Parameters:
    - final_rewards: Dictionary containing final rewards for each model.
    - models: List of model names to compare.
    - base_model: The model to compare others against (default: 'SEC').
    
    Returns:
    - results_df: Pandas DataFrame containing the statistical test results.
    """
    results = {
        'Model': [],
        'Mean (Base)': [],
        'Mean (Comparison)': [],
        '95% CI Lower (Base)': [],
        '95% CI Upper (Base)': [],
        '95% CI Lower (Comparison)': [],
        '95% CI Upper (Comparison)': [],
        'p-value': [],
        'Significant': []
    }

    # Base model rewards
    base_trureward = data[base_model]['abs_tru_reward']
    base_trureward = np.asarray(base_trureward, dtype=np.float32)
    print("base_truerward_sum ", np.sum(base_trureward))

    if np.sum(base_trureward) > 0:
        if len(base_trureward[0]) > 5000: 
            based_data_half = np.hsplit(base_trureward, 2)
            base_trureward = based_data_half[0]

    base_data = base_trureward[:, -1]
    base_mean = np.mean(base_data)
    base_std = np.std(base_data)
    base_sem = base_std / math.sqrt(len(base_data))
    base_ci_lower = base_mean - 1.96 * base_sem
    base_ci_upper = base_mean + 1.96 * base_sem
    
    print("reward mean last ", base_mean)
    print("reward std last ", base_std)
    print("reward sem last", base_sem)
    
    for model in models:
        if model == base_model:
            continue  # Skip comparison with itself

        model_trureward = data[model]['abs_tru_reward']
        model_trureward = np.asarray(model_trureward, dtype=np.float32)
        print("model_truerward_sum ", np.sum(model_trureward))

        if np.sum(model_trureward) > 0:
            if len(model_trureward[0]) > 5000: 
                model_data_half = np.hsplit(model_trureward, 2)
                model_trureward = model_data_half[0]

        comp_data  = model_trureward[:, -1]
        comp_mean = np.mean(comp_data)
        comp_std = np.std(comp_data)
        comp_sem = comp_std / math.sqrt(len(comp_data))
        comp_ci_lower = comp_mean - 1.96 * comp_sem
        comp_ci_upper = comp_mean + 1.96 * comp_sem

        print("comp reward mean last ", comp_mean)
        print("comp reward std last ", comp_std)
        print("comp reward sem last", comp_sem)
        
        # Perform Mann-Whitney U test
        stat, p_val = mannwhitneyu(base_data, comp_data)

        # Determine significance
        alpha = 0.05  # Significance level
        significant = 'Yes' if p_val < alpha else 'No'

        # Populate results
        results['Model'].append(model)
        results['Mean (Base)'].append(base_mean)
        results['Mean (Comparison)'].append(comp_mean)
        results['95% CI Lower (Base)'].append(base_ci_lower)
        results['95% CI Upper (Base)'].append(base_ci_upper)
        results['95% CI Lower (Comparison)'].append(comp_ci_lower)
        results['95% CI Upper (Comparison)'].append(comp_ci_upper)
        results['p-value'].append(p_val)
        results['Significant'].append(significant)

    # Create DataFrame
    results_df = pd.DataFrame(results)
    return results_df

def apply_bonferroni_correction(results_df):
    """
    Apply Bonferroni correction to p-values in the results DataFrame.
    
    Parameters:
    - results_df: DataFrame with statistical test results, including p-values.
    
    Returns:
    - results_df: Updated DataFrame with Bonferroni-adjusted p-values.
    """
    num_comparisons = len(results_df)
    results_df['Adjusted p-value (Bonferroni)'] = results_df['p-value'] * num_comparisons
    results_df['Adjusted p-value (Bonferroni)'] = results_df['Adjusted p-value (Bonferroni)'].clip(upper=1.0)
    return results_df

def display_statistical_results(results_df):
    """
    Display the statistical test results in a formatted table.
    
    Parameters:
    - results_df: Pandas DataFrame containing the statistical test results.
    """
    # Format p-values
    results_df['p-value'] = results_df['p-value'].apply(lambda x: f"{x:.4f}")
    
    # Display the table
    print("\n**Statistical Test Results (Base Model: SEC)**\n")
    display(results_df.style.hide_index().set_table_styles(
        [{
            'selector': 'th',
            'props': [('font-size', '12pt'), ('text-align', 'center')]
        }]
    ).set_properties(**{
        'font-size': '12pt',
        'text-align': 'center'
    }))
    
    # Optionally, save to CSV
    # results_df.to_csv('statistical_test_results.csv', index=False)

def main_statistical_analysis(data, models):
    """
    Main function to perform extraction of final rewards, statistical testing, and display results.
    
    Parameters:
    - data: Dictionary containing the reward data for each model.
    - models: List of model names to compare.
    """
    # Extract final rewards from the last episodes
    #final_rewards = extract_final_rewards(data, models, end_window_size=100)

    # Perform pairwise statistical tests
    results_df = perform_statistical_tests(data, models, base_model='SEC')
    
    # Apply Bonferroni correction for multiple comparisons
    results_df = apply_bonferroni_correction(results_df)
    
    # Display results
    display_statistical_results(results_df)
    
    # Optional: Save the table as a CSV for inclusion in the manuscript
    # results_df.to_csv(plot_path + 'new_statistical_test_results.csv', index=False)
    results_df.to_csv(plot_path + task + '_statistical_test_results.csv', index=False)

In [58]:
# Example usage:
task = 'DoubleTMaze'

# Define file paths
file_path = '../data/exp1_2tmaze/'
plot_path = '../data/exp1_2tmaze/'

# Load data
data = load_plot_data('data_v3', file_path)  # Ensure this function returns the correct structure

# Define models and labels
models = ['SEC', 'NSEC', 'MFEC-ae', 'MFEC-rp', 'DQN', 'ERLAM']
#labels = ['SEC', 'NSEC', 'MFEC-ae', 'MFEC-rp', 'DQN', 'ERLAM']
    
main_statistical_analysis(data, models)

base_truerward_sum  232097.61
reward mean last  2.78515
reward std last  0.124701716
reward sem last 0.027884151389984718
model_truerward_sum  100490.64
comp reward mean last  1.25415
comp reward std last  1.2728102
comp reward sem last 0.2846090175982817
model_truerward_sum  70509.586
comp reward mean last  1.0884
comp reward std last  1.33557
comp reward sem last 0.2986425258979946
model_truerward_sum  22324.096
comp reward mean last  0.717
comp reward std last  1.2418815
comp reward sem last 0.3585003062054628
model_truerward_sum  62076.54
comp reward mean last  1.9991
comp reward std last  1.3089341
comp reward sem last 0.4139213039414039
model_truerward_sum  78262.09
comp reward mean last  1.5838001
comp reward std last  1.3031106
comp reward sem last 0.4120797537583833

**Statistical Test Results (Base Model: SEC)**



C:\Users\zerat\AppData\Local\Temp\ipykernel_18988\305117225.py:141: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(results_df.style.hide_index().set_table_styles(


Model,Mean (Base),Mean (Comparison),95% CI Lower (Base),95% CI Upper (Base),95% CI Lower (Comparison),95% CI Upper (Comparison),p-value,Significant,Adjusted p-value (Bonferroni)
NSEC,2.785150,1.254150,2.730497,2.839803,0.696316,1.811984,0.0000,Yes,0.000005
MFEC-ae,2.785150,1.088400,2.730497,2.839803,0.503061,1.673739,0.0000,Yes,0.000163
MFEC-rp,2.785150,0.717000,2.730497,2.839803,0.014339,1.419661,0.0117,Yes,0.058678
DQN,2.785150,1.999100,2.730497,2.839803,1.187814,2.810386,0.6281,No,1.000000
ERLAM,2.785150,1.583800,2.730497,2.839803,0.776124,2.391476,0.0077,Yes,0.038437


## Statistics v2

In [158]:
import numpy as np
import math
import pandas as pd
from scipy.stats import mannwhitneyu
import itertools
import matplotlib.pyplot as plt

def extract_final_rewards(data, models, end_window_size=100):
    """
    Extract final rewards from the last `end_window_size` episodes for each model.
    
    Parameters:
    - data: Dictionary containing reward data for each model.
    - models: List of model names to extract.
    - end_window_size: Number of episodes to consider from the end.
    
    Returns:
    - final_rewards: Dictionary with final rewards for each model.
    """
    final_rewards = {model: data[model]['abs_tru_reward'][:, -end_window_size:].mean(axis=1) for model in models}
    return final_rewards

def perform_statistical_tests(final_rewards, models, base_model='SEC'):
    """
    Perform pairwise Mann-Whitney U tests comparing the base_model against other models.
    
    Parameters:
    - final_rewards: Dictionary containing final rewards for each model.
    - models: List of model names to compare.
    - base_model: The model to compare others against (default: 'SEC').
    
    Returns:
    - results_df: Pandas DataFrame containing the statistical test results.
    """
    results = {
        'Model': [],
        'Mean': [],
        '95% CI Lower': [],
        '95% CI Upper': [],
        'p-value': [],
        'Significant': []
    }

    # Base model rewards
    base_trureward = data[base_model]['abs_tru_reward']
    base_trureward = np.asarray(base_trureward, dtype=np.float32)
    print("base_truerward_sum ", np.sum(base_trureward))

    if np.sum(base_trureward) > 0:
        if len(base_trureward[0]) > 5000: 
            based_data_half = np.hsplit(base_trureward, 2)
            base_trureward = based_data_half[0]

    base_data = base_trureward[:, -1]
    base_mean = np.mean(base_data)
    base_std = np.std(base_data)
    base_sem = base_std / math.sqrt(len(base_data))
    base_ci_lower = base_mean - 1.96 * base_sem
    base_ci_upper = base_mean + 1.96 * base_sem
    
    print("reward mean last ", base_mean)
    print("reward std last ", base_std)
    print("reward sem last", base_sem)
    
    results['Model'].append(base_model)
    results['Mean'].append(base_mean)
    results['95% CI Lower'].append(base_ci_lower)
    results['95% CI Upper'].append(base_ci_upper)
    results['p-value'].append(None)  # No p-value for the base model
    results['Significant'].append(None)  # No significance comparison for the base model
    
    for model in models:
        if model == base_model:
            continue  # Skip comparison with itself

        model_trureward = data[model]['abs_tru_reward']
        model_trureward = np.asarray(model_trureward, dtype=np.float32)
        print("model_truerward_sum ", np.sum(model_trureward))

        if np.sum(model_trureward) > 0:
            if len(model_trureward[0]) > 5000: 
                model_data_half = np.hsplit(model_trureward, 2)
                model_trureward = model_data_half[0]

        comp_data  = model_trureward[:, -1]
        comp_mean = np.mean(comp_data)
        comp_std = np.std(comp_data)
        comp_sem = comp_std / math.sqrt(len(comp_data))
        comp_ci_lower = comp_mean - 1.96 * comp_sem
        comp_ci_upper = comp_mean + 1.96 * comp_sem

        print("comp reward mean last ", comp_mean)
        print("comp reward std last ", comp_std)
        print("comp reward sem last", comp_sem)
        
        # Perform Mann-Whitney U test
        stat, p_val = mannwhitneyu(base_data, comp_data)

        # Determine significance
        alpha = 0.05  # Significance level
        significant = 'Yes' if p_val < alpha else 'No'

        # Populate results
        results['Model'].append(model)
        results['Mean'].append(comp_mean)
        results['95% CI Lower'].append(comp_ci_lower)
        results['95% CI Upper'].append(comp_ci_upper)
        results['p-value'].append(p_val)
        results['Significant'].append(significant)
        
    # Create DataFrame
    results_df = pd.DataFrame(results)
    return results_df

def apply_bonferroni_correction(results_df):
    """
    Apply Bonferroni correction to p-values in the results DataFrame.
    
    Parameters:
    - results_df: DataFrame with statistical test results, including p-values.
    
    Returns:
    - results_df: Updated DataFrame with Bonferroni-adjusted p-values.
    """
    num_comparisons = len(results_df)
    results_df['Adjusted p-value (Bonferroni)'] = results_df['p-value'] * num_comparisons
    results_df['Adjusted p-value (Bonferroni)'] = results_df['Adjusted p-value (Bonferroni)'].clip(upper=1.0)
    return results_df

def display_statistical_results(results_df):
    """
    Display the statistical test results in a formatted table.
    
    Parameters:
    - results_df: Pandas DataFrame containing the statistical test results.
    """
    # Format p-values
    results_df['p-value'] = results_df['p-value'].apply(lambda x: f"{x:.4f}" if pd.notna(x) else 'N/A')
    
    # Display the table
    print("\n**Statistical Test Results (Base Model: SEC)**\n")
    display(results_df.style.hide_index().set_table_styles(
        [{
            'selector': 'th',
            'props': [('font-size', '12pt'), ('text-align', 'center')]
        }]
    ).set_properties(**{
        'font-size': '12pt',
        'text-align': 'center'
    }))
    
    # Optionally, save to CSV
    # results_df.to_csv('statistical_test_results.csv', index=False)

def main_statistical_analysis(data, models, base_model='SEC'):
    """
    Main function to perform extraction of final rewards, statistical testing, and display results.
    
    Parameters:
    - data: Dictionary containing the reward data for each model.
    - models: List of model names to compare.
    """
    # Extract final rewards from the last episodes
    #final_rewards = extract_final_rewards(data, models, end_window_size=100)

    # Perform pairwise statistical tests
    results_df = perform_statistical_tests(data, models, base_model=base_model)
    
    # Apply Bonferroni correction for multiple comparisons
    results_df = apply_bonferroni_correction(results_df)
    
    # Display results
    display_statistical_results(results_df)
    
    # Optional: Save the table as a CSV for inclusion in the manuscript
    results_df.to_csv(plot_path + task + '_statistical_test_results.csv', index=False)


# Statistics v3

In [7]:
import numpy as np
import math
import pandas as pd
from scipy.stats import mannwhitneyu
import itertools
import matplotlib.pyplot as plt

def extract_final_rewards(data, models, end_window_size=100):
    """
    Extract final rewards from the last `end_window_size` episodes for each model.
    
    Parameters:
    - data: Dictionary containing reward data for each model.
    - models: List of model names to extract.
    - end_window_size: Number of episodes to consider from the end.
    
    Returns:
    - final_rewards: Dictionary with final rewards for each model.
    """
    final_rewards = {model: data[model]['abs_tru_reward'][:, -end_window_size:].mean(axis=1) for model in models}
    return final_rewards

def perform_statistical_tests(final_rewards, models, base_model='SEC'):
    """
    Perform pairwise Mann-Whitney U tests comparing the base_model against other models.
    
    Parameters:
    - final_rewards: Dictionary containing final rewards for each model.
    - models: List of model names to compare.
    - base_model: The model to compare others against (default: 'SEC').
    
    Returns:
    - results_df: Pandas DataFrame containing the statistical test results.
    """
    results = {
        'Model': [],
        'Mean': [],
        '95% CI Lower': [],
        '95% CI Upper': [],
        'p-value': [],
        'Significant': []
    }

    # Base model rewards
    base_trureward = data[base_model]['abs_tru_reward']
    base_trureward = np.asarray(base_trureward, dtype=np.float32)
    print("base_truerward_sum ", np.sum(base_trureward))

    if np.sum(base_trureward) > 0:
        if len(base_trureward[0]) > 5000: 
            based_data_half = np.hsplit(base_trureward, 2)
            base_trureward = based_data_half[0]

    base_data = np.sum(base_trureward, axis=1)
    base_mean = np.mean(base_data)
    base_std = np.std(base_data)
    base_sem = base_std / math.sqrt(len(base_data))
    base_ci_lower = base_mean - 1.96 * base_sem
    base_ci_upper = base_mean + 1.96 * base_sem
    
    print("total reward mean ", base_mean)
    print("total reward std ", base_std)
    print("total reward sem", base_sem)
    
    results['Model'].append(base_model)
    results['Mean'].append(base_mean)
    results['95% CI Lower'].append(base_ci_lower)
    results['95% CI Upper'].append(base_ci_upper)
    results['p-value'].append(None)  # No p-value for the base model
    results['Significant'].append(None)  # No significance comparison for the base model
    
    for model in models:
        if model == base_model:
            continue  # Skip comparison with itself

        model_trureward = data[model]['abs_tru_reward']
        model_trureward = np.asarray(model_trureward, dtype=np.float32)
        print("model_truerward_sum ", np.sum(model_trureward))

        if np.sum(model_trureward) > 0:
            if len(model_trureward[0]) > 5000: 
                model_data_half = np.hsplit(model_trureward, 2)
                model_trureward = model_data_half[0]

        comp_data = np.sum(model_trureward, axis=1)
        comp_mean = np.mean(comp_data)
        comp_std = np.std(comp_data)
        comp_sem = comp_std / math.sqrt(len(comp_data))
        comp_ci_lower = comp_mean - 1.96 * comp_sem
        comp_ci_upper = comp_mean + 1.96 * comp_sem

        print("total comp reward mean ", comp_mean)
        print("total comp reward std ", comp_std)
        print("total comp reward sem", comp_sem)
        
        # Perform Mann-Whitney U test
        stat, p_val = mannwhitneyu(base_data, comp_data)

        # Determine significance
        alpha = 0.05  # Significance level
        significant = 'Yes' if p_val < alpha else 'No'

        # Populate results
        results['Model'].append(model)
        results['Mean'].append(comp_mean)
        results['95% CI Lower'].append(comp_ci_lower)
        results['95% CI Upper'].append(comp_ci_upper)
        results['p-value'].append(p_val)
        results['Significant'].append(significant)
        
    # Create DataFrame
    results_df = pd.DataFrame(results)
    return results_df

def apply_bonferroni_correction(results_df):
    """
    Apply Bonferroni correction to p-values in the results DataFrame.
    
    Parameters:
    - results_df: DataFrame with statistical test results, including p-values.
    
    Returns:
    - results_df: Updated DataFrame with Bonferroni-adjusted p-values.
    """
    num_comparisons = len(results_df)
    results_df['Adjusted p-value (Bonferroni)'] = results_df['p-value'] * num_comparisons
    results_df['Adjusted p-value (Bonferroni)'] = results_df['Adjusted p-value (Bonferroni)'].clip(upper=1.0)
    return results_df

def display_statistical_results(results_df):
    """
    Display the statistical test results in a formatted table.
    
    Parameters:
    - results_df: Pandas DataFrame containing the statistical test results.
    """
    # Format p-values
    results_df['p-value'] = results_df['p-value'].apply(lambda x: f"{x:.4f}" if pd.notna(x) else 'N/A')
    
    # Display the table
    print("\n**Statistical Test Results (Base Model: SEC)**\n")
    display(results_df.style.hide_index().set_table_styles(
        [{
            'selector': 'th',
            'props': [('font-size', '12pt'), ('text-align', 'center')]
        }]
    ).set_properties(**{
        'font-size': '12pt',
        'text-align': 'center'
    }))
    
    # Optionally, save to CSV
    # results_df.to_csv('statistical_test_results.csv', index=False)

def main_statistical_analysis(data, models, base_model='SEC'):
    """
    Main function to perform extraction of final rewards, statistical testing, and display results.
    
    Parameters:
    - data: Dictionary containing the reward data for each model.
    - models: List of model names to compare.
    """
    # Extract final rewards from the last episodes
    #final_rewards = extract_final_rewards(data, models, end_window_size=100)

    # Perform pairwise statistical tests
    results_df = perform_statistical_tests(data, models, base_model=base_model)
    
    # Apply Bonferroni correction for multiple comparisons
    results_df = apply_bonferroni_correction(results_df)
    
    # Display results
    display_statistical_results(results_df)
    
    # Optional: Save the table as a CSV for inclusion in the manuscript
    results_df.to_csv(plot_path + task + '_statistical_test_results.csv', index=False)


### For exp 1

In [185]:
# Example usage:
task = 'DoubleTMaze'

# Define file paths
file_path = '../data/exp1_2tmaze/'
plot_path = '../data/exp1_2tmaze/'

# Load data
data = load_plot_data('data_v3', file_path)  # Ensure this function returns the correct structure
data.keys()

# Define models and labels
models_tmaze = ['SEC', 'NSEC', 'MFEC-ae', 'MFEC-rp', 'DQN', 'ERLAM']
models_cylinder = ['SEC_500', 'NSEC_500', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'DQN_25k', 'ERLAM_5k']
models_detour = ['SEC_500', 'NSEC_500', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'DQN_25k', 'ERLAM_25k']
models_permanence = ['SEC_500', 'NSEC_500', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'DQN_25k', 'ERLAM_5k']
#labels = ['SEC', 'NSEC', 'MFEC-ae', 'MFEC-rp', 'DQN', 'ERLAM']
    
main_statistical_analysis(data, models_tmaze, base_model='SEC')

base_truerward_sum  232097.61
total reward mean  11604.881
total reward std  1649.4579
total reward sem 368.8299958542612
model_truerward_sum  100490.64
total comp reward mean  5024.5312
total comp reward std  2889.6023
total comp reward sem 646.1347159384708
model_truerward_sum  70509.586
total comp reward mean  3525.4792
total comp reward std  2808.5393
total comp reward sem 628.0084807128563
model_truerward_sum  22324.096
total comp reward mean  1860.3412
total comp reward std  3041.2249
total comp reward sem 877.9259939217131
model_truerward_sum  62076.54
total comp reward mean  6207.6543
total comp reward std  4202.738
total comp reward sem 1329.0223834250437
model_truerward_sum  78262.09
total comp reward mean  7826.2095
total comp reward std  2651.4067
total comp reward sem 838.4484296486705

**Statistical Test Results (Base Model: SEC)**



C:\Users\zerat\AppData\Local\Temp\ipykernel_18988\2656814379.py:142: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(results_df.style.hide_index().set_table_styles(


Model,Mean,95% CI Lower,95% CI Upper,p-value,Significant,Adjusted p-value (Bonferroni)
SEC,11604.880859,10881.974068,12327.787651,N/A,None,nan
NSEC,5024.531250,3758.107207,6290.955293,0.0000,Yes,0.000002
MFEC-ae,3525.479248,2294.582626,4756.375870,0.0000,Yes,0.000001
MFEC-rp,1860.341187,139.606238,3581.076135,0.0000,Yes,0.000035
DQN,6207.654297,3602.770425,8812.538168,0.0002,Yes,0.001207
ERLAM,7826.209473,6182.850551,9469.568395,0.0002,Yes,0.001435


### For exp 2

In [204]:
# Example usage:
task = 'Memory Constraints'

# Define file paths
file_path = '../data/exp2_memory_constraints/'
plot_path = '../data/exp2_memory_constraints/'

# Load data
data = load_plot_data('data_true', file_path)  # Ensure this function returns the correct structure
data.keys()


# Define models and labels
#models_SEC = ['SEC_ltm125', 'SEC_ltm250', 'SEC_ltm500', 'SEC_ltm1000']
#models_NSEC = ['NSEC_ltm125', 'NSEC_ltm250', 'NSEC_ltm500', 'NSEC_ltm1000']
    
#main_statistical_analysis(data, models_SEC, base_model='SEC_ltm1000')

dict_keys(['SEC_ltm125', 'SEC_ltm250', 'SEC_ltm500', 'SEC_ltm1000', 'NSEC_ltm125', 'NSEC_ltm250', 'NSEC_ltm500', 'NSEC_ltm1000'])

In [206]:
# Example usage:
task = 'Memory Constraints SEC'

# Define file paths
file_path = '../data/exp2_memory_constraints/'
plot_path = '../data/exp2_memory_constraints/'

# Load data
data = load_plot_data('data_true', file_path)  # Ensure this function returns the correct structure
data.keys()

# Define models and labels
models_SEC = ['SEC_ltm125', 'SEC_ltm250', 'SEC_ltm500', 'SEC_ltm1000']
#models_NSEC = ['NSEC_ltm125', 'NSEC_ltm250', 'NSEC_ltm500', 'NSEC_ltm1000']
    
main_statistical_analysis(data, models_SEC, base_model='SEC_ltm1000')

base_truerward_sum  513458.06
total reward mean  12174.311
total reward std  843.6874
total reward sem 188.6542328809337
model_truerward_sum  184826.64
total comp reward mean  9241.334
total comp reward std  1812.4886
total comp reward sem 405.28478241693074
model_truerward_sum  207027.78
total comp reward mean  10351.388
total comp reward std  2054.2239
total comp reward sem 459.3384229870351
model_truerward_sum  500778.75
total comp reward mean  12031.594
total comp reward std  1303.3646
total comp reward sem 291.44118987848617

**Statistical Test Results (Base Model: SEC)**



C:\Users\zerat\AppData\Local\Temp\ipykernel_18988\2656814379.py:142: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(results_df.style.hide_index().set_table_styles(


Model,Mean,95% CI Lower,95% CI Upper,p-value,Significant,Adjusted p-value (Bonferroni)
SEC_ltm1000,12174.310547,11804.548250,12544.072843,N/A,None,nan
SEC_ltm125,9241.333984,8446.975811,10035.692158,0.0000,Yes,0.000009
SEC_ltm250,10351.387695,9451.084386,11251.691004,0.0023,Yes,0.009365
SEC_ltm500,12031.593750,11460.369018,12602.818482,0.7972,No,1.000000


In [207]:
# Example usage:
task = 'Memory Constraints NSEC'

# Define file paths
file_path = '../data/exp2_memory_constraints/'
plot_path = '../data/exp2_memory_constraints/'

# Load data
data = load_plot_data('data_true', file_path)  # Ensure this function returns the correct structure
data.keys()

# Define models and labels
#models_SEC = ['SEC_ltm125', 'SEC_ltm250', 'SEC_ltm500', 'SEC_ltm1000']
models_NSEC = ['NSEC_ltm125', 'NSEC_ltm250', 'NSEC_ltm500', 'NSEC_ltm1000']
    
main_statistical_analysis(data, models_NSEC, base_model='NSEC_ltm1000')

base_truerward_sum  148484.38
total reward mean  7424.2197
total reward std  3074.0898
total reward sem 687.3873859566706
model_truerward_sum  80159.71
total comp reward mean  3817.1301
total comp reward std  2194.6387
total comp reward sem 478.90942080688285
model_truerward_sum  103730.336
total comp reward mean  5186.517
total comp reward std  2860.7925
total comp reward sem 639.6926455848364
model_truerward_sum  123633.234
total comp reward mean  6181.6616
total comp reward std  2764.6687
total comp reward sem 618.1987151086365

**Statistical Test Results (Base Model: SEC)**



C:\Users\zerat\AppData\Local\Temp\ipykernel_18988\2656814379.py:142: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(results_df.style.hide_index().set_table_styles(


Model,Mean,95% CI Lower,95% CI Upper,p-value,Significant,Adjusted p-value (Bonferroni)
NSEC_ltm1000,7424.219727,6076.940450,8771.499003,N/A,None,nan
NSEC_ltm125,3817.130127,2878.467662,4755.792592,0.0007,Yes,0.002659
NSEC_ltm250,5186.517090,3932.719504,6440.314675,0.0411,Yes,0.164494
NSEC_ltm500,6181.661621,4969.992139,7393.331103,0.2733,No,1.000000


### For exp 3

In [193]:
# Example usage:
task = '1ablation'

# Define file paths
file_path = '../data/exp3_ablations/'
plot_path = '../data/exp3_ablations/'

# Load data
data = load_plot_data('data_new', file_path)  # Ensure this function returns the correct structure
data.keys()

# Define models and labels
#models_tmaze = ['SEC', 'NSEC', 'MFEC-ae', 'MFEC-rp', 'DQN', 'ERLAM']
#models_cylinder = ['SEC_500', 'NSEC_500', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'DQN_25k', 'ERLAM_5k']
#models_detour = ['SEC_500', 'NSEC_500', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'DQN_25k', 'ERLAM_25k']
#models_permanence = ['SEC_500', 'NSEC_500', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'DQN_25k', 'ERLAM_5k']

models_1ablation = ['noDist', 'noRR', 'noGi_new', 'default']
models_2ablation = ['soloDist', 'soloRR', 'soloGi', 'default']

#labels = ['SEC', 'NSEC', 'MFEC-ae', 'MFEC-rp', 'DQN', 'ERLAM']
    
main_statistical_analysis(data, models_1ablation, base_model='default')

base_truerward_sum  165764.45
total reward mean  11840.318
total reward std  1485.1364
total reward sem 396.919385988878
model_truerward_sum  140945.31
total comp reward mean  10841.947
total comp reward std  1985.7986
total comp reward sem 550.7614320999575
model_truerward_sum  160277.66
total comp reward mean  11448.405
total comp reward std  1565.0648
total comp reward sem 418.2811672891668
model_truerward_sum  162697.52
total comp reward mean  11621.252
total comp reward std  1384.2921
total comp reward sem 369.96762962611865

**Statistical Test Results (Base Model: SEC)**



C:\Users\zerat\AppData\Local\Temp\ipykernel_18988\2656814379.py:142: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(results_df.style.hide_index().set_table_styles(


Model,Mean,95% CI Lower,95% CI Upper,p-value,Significant,Adjusted p-value (Bonferroni)
default,11840.318359,11062.356363,12618.280356,N/A,None,nan
noDist,10841.947266,9762.454859,11921.439673,0.1523,No,0.609123
noRR,11448.405273,10628.574186,12268.236361,0.4213,No,1.000000
noGi_new,11621.251953,10896.115399,12346.388507,0.5053,No,1.000000


### For exp 4

In [11]:
# Example usage:
task = 'SECmemory'

# Define file paths
file_path = '../data/exp4_forgetting/'
plot_path = '../data/exp4_forgetting/'

# Load data
data = load_plot_data('data_new', file_path)  # Ensure this function returns the correct structure
data.keys()

# Define models and labels
#models_tmaze = ['SEC', 'NSEC', 'MFEC-ae', 'MFEC-rp', 'DQN', 'ERLAM']
#models_cylinder = ['SEC_500', 'NSEC_500', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'DQN_25k', 'ERLAM_5k']
#models_detour = ['SEC_500', 'NSEC_500', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'DQN_25k', 'ERLAM_25k']
#models_permanence = ['SEC_500', 'NSEC_500', 'MFEC_VAE_100K', 'MFEC_RP_100K', 'DQN_25k', 'ERLAM_5k']

models_sec = ['SEC', 'SEC-PRIOR', 'SEC-FIFO_v2']
models_nsec = ['NSEC', 'NSEC-FIFO', 'NSEC-RWD']

#labels = ['SEC', 'NSEC', 'MFEC-ae', 'MFEC-rp', 'DQN', 'ERLAM']
    
main_statistical_analysis(data, models_nsec, base_model='NSEC')

base_truerward_sum  100490.64
total reward mean  5024.5312
total reward std  2889.6023
total reward sem 646.1347159384708
model_truerward_sum  103469.84
total comp reward mean  5173.492
total comp reward std  3293.304
total comp reward sem 736.40515141236
model_truerward_sum  133381.92
total comp reward mean  6669.096
total comp reward std  2805.5818
total comp reward sem 627.3471592411905

**Statistical Test Results (Base Model: SEC)**



C:\Users\zerat\AppData\Local\Temp\ipykernel_9924\2656814379.py:142: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(results_df.style.hide_index().set_table_styles(


Model,Mean,95% CI Lower,95% CI Upper,p-value,Significant,Adjusted p-value (Bonferroni)
NSEC,5024.531250,3758.107207,6290.955293,N/A,None,nan
NSEC-FIFO,5173.492188,3730.138091,6616.846284,0.9246,No,1.000000
NSEC-RWD,6669.096191,5439.495759,7898.696624,0.0909,No,0.272722


## Integrated table exp1

In [208]:
import pandas as pd

file_path = '../data/'

# Load the CSV files
double_tmaze_df = pd.read_csv(file_path+'DoubleTMaze_statistical_test_results.csv')
cylinder_df = pd.read_csv(file_path+'Cylinder_statistical_test_results.csv')
detour_df = pd.read_csv(file_path+'Detour_statistical_test_results.csv')
permanence_df = pd.read_csv(file_path+'Permanence_statistical_test_results.csv')

# Define the function to generate the comparison table
def generate_comparison_table_v3(task_dfs, task_names, models, base_model='SEC'):
    comparison_data = []
    
    # Iterate through each task's dataframe
    for task_df, task_name in zip(task_dfs, task_names):
        task_data = {'Task': task_name}
        
        # Add the mean performance for each model and format accordingly
        base_mean = task_df[task_df['Model'] == base_model]['Mean'].values[0]
        best_model = None
        best_mean = -float('inf')

        # Find the best performing model
        for model in models:
            model_mean = task_df[task_df['Model'] == model]['Mean'].values[0]
            if model_mean > best_mean:
                best_mean = model_mean
                best_model = model

        # Populate comparison data, with formatting
        for model in models:
            model_mean = task_df[task_df['Model'] == model]['Mean'].values[0]
            is_significant = task_df[task_df['Model'] == model]['Significant'].values[0] == 'No'

            if model == best_model:
                # Highlight best performing model (bold + underline)
                task_data[model] = f"**__{model_mean:.2f}__**"
            elif not is_significant:
                # Highlight models not statistically significant from SEC (bold)
                task_data[model] = f"**{model_mean:.2f}**"
            else:
                # Normal formatting for significant models
                task_data[model] = f"{model_mean:.2f}"
        
        comparison_data.append(task_data)
    
    # Create DataFrame from comparison data
    comparison_df = pd.DataFrame(comparison_data)
    return comparison_df

# Define task names and models
task_names = ['Double T-Maze', 'Cylinder', 'Detour', 'Object Permanence']
models = ['SEC', 'NSEC', 'MFEC-ae', 'MFEC-rp', 'DQN', 'ERLAM']

# Generate the comparison table
comparison_df_v3 = generate_comparison_table_v3(
    task_dfs=[double_tmaze_df, cylinder_df, detour_df, permanence_df],
    task_names=task_names,
    models=models
)

# Display the table (replace with your display mechanism)
print(comparison_df_v3)


                Task               SEC         NSEC      MFEC-ae      MFEC-rp  \
0      Double T-Maze  **__11604.88__**  **5024.53**  **3525.48**  **1860.34**   
1           Cylinder   **__5829.88__**      5130.67      4065.24      5254.58   
2             Detour   **__1181.82__**     **3.48**     **3.74**     **4.02**   
3  Object Permanence   **__7224.33__**  **3803.15**  **5218.06**  **4904.88**   

           DQN        ERLAM  
0  **6207.65**  **7826.21**  
1  **3490.08**  **4665.28**  
2     **8.03**    **24.52**  
3  **3850.78**  **3793.70**  


In [18]:
def display_performance_comparison(results_df):
    """
    Display the performance comparison results in a formatted table.
    
    Parameters:
    - results_df: Pandas DataFrame containing the model performance comparison.
    """
    # Display the table with formatting similar to your previous example
    print("\n**Model Performance Comparison**\n")
    
    # Format and display the DataFrame with styled output
    display(results_df.style.hide_index().set_table_styles(
        [   
            # Style for the header
            {'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center')]},
            # Style for the body
            {'selector': 'td', 'props': [('font-size', '12pt'), ('text-align', 'center')]}
        ]
    ).set_properties(**{
        'text-align': 'center'
    }))
    
def display_and_save_dataframe_v3(df, filename="comparison_results.csv"):
    """
    Displays the DataFrame using a custom formatting function and saves it to a CSV file.
    
    Parameters:
    - df: The DataFrame to display and save.
    - filename: The name of the CSV file to save the DataFrame (default is 'comparison_results.csv').
    """
    # Display the DataFrame using the custom display function
    display_performance_comparison(df)
    
    # Save the dataframe to a CSV file
    df.to_csv(filename, index=False)

# Call the function to display and save the dataframe
display_and_save_dataframe_v3(comparison_df_v3, filename="consistent_model_performance_comparison.csv")


## Integrated table exp3

In [210]:
# Adjust the function to handle different model sets for each task separately
def generate_comparison_table_v5(task_dfs, task_names, model_lists, base_model='SEC'):
    comparison_data = []
    
    # Iterate through each task's dataframe and corresponding model list
    for task_df, task_name, models in zip(task_dfs, task_names, model_lists):
        task_data = {'Task': task_name}
        
        # Add the mean performance for each model and format accordingly
        base_mean = task_df[task_df['Model'] == base_model]['Mean'].values[0]
        best_model = None
        best_mean = -float('inf')

        # Find the best performing model
        for model in models:
            model_mean = task_df[task_df['Model'] == model]['Mean'].values[0]
            if model_mean > best_mean:
                best_mean = model_mean
                best_model = model

        # Populate comparison data, with formatting
        for model in models:
            model_mean = task_df[task_df['Model'] == model]['Mean'].values[0]
            is_significant = task_df[task_df['Model'] == model]['Significant'].values[0] == 'No'

            if model == best_model:
                # Highlight best performing model (bold + underline)
                task_data[model] = f"**__{model_mean:.2f}__**"
            elif not is_significant:
                # Highlight models not statistically significant from SEC (bold)
                task_data[model] = f"**{model_mean:.2f}**"
            else:
                # Normal formatting for significant models
                task_data[model] = f"{model_mean:.2f}"
        
        comparison_data.append(task_data)
    
    # Create DataFrame from comparison data
    comparison_df = pd.DataFrame(comparison_data)
    return comparison_df


In [211]:
file_path = '../data/'

# Load the CSV files
ablation_1_df = pd.read_csv(file_path+'1ablation_statistical_test_results.csv')
ablation_2_df = pd.read_csv(file_path+'2ablation_statistical_test_results.csv')
task_names = ['Ablation Study 1', 'Ablation Study 2']
models_1ablation = ['SEC', 'noDist', 'noRR', 'noGi']
models_2ablation = ['SEC', 'soloDist', 'soloRR', 'soloGi']

#print(ablation_1_df['Model'].unique())
#print(ablation_2_df['Model'].unique())

# Generate the comparison table with the updated function
comparison_df_v5 = generate_comparison_table_v5(
    task_dfs=[ablation_1_df, ablation_2_df],
    task_names=task_names,
    model_lists=[models_1ablation, models_2ablation]
)

In [17]:
from IPython.display import display

def display_performance_comparison(results_df):
    """
    Display the performance comparison results in a formatted table.
    
    Parameters:
    - results_df: Pandas DataFrame containing the model performance comparison.
    """
    # Display the table with formatting
    print("\n**Model Performance Comparison**\n")
    
    # Format and display the DataFrame with styled output
    display(results_df.style.hide_index().set_table_styles(
        [   
            # Style for the header
            {'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center')]},
            # Style for the body
            {'selector': 'td', 'props': [('font-size', '12pt'), ('text-align', 'center')]}
        ]
    ).set_properties(**{
        'text-align': 'center'
    }))
    
def display_and_save_dataframe_v3(df, filename="comparison_results.csv"):
    """
    Displays the DataFrame using a custom formatting function and saves it to a CSV file.
    
    Parameters:
    - df: The DataFrame to display and save.
    - filename: The name of the CSV file to save the DataFrame (default is 'comparison_results.csv').
    """
    # Display the DataFrame using the custom display function
    display_performance_comparison(df)
    
    # Save the dataframe to a CSV file
    df.to_csv(filename, index=False)

# Call the function to display and save the dataframe
display_and_save_dataframe_v3(comparison_df_v5, filename="ablation_studies_performance_comparison.csv")


NameError: name 'comparison_df_v5' is not defined

## Integrated table exp4

In [12]:
# Modify the generate_comparison_table_v5 function to accept different base models for each table
def generate_two_comparison_tables(task_dfs, task_names, model_lists, base_models):
    """
    Generate two comparison tables with different base models for each dataset.

    Parameters:
    - task_dfs: List of DataFrames corresponding to the tasks.
    - task_names: List of task names.
    - model_lists: List of model names for each task.
    - base_models: List of base models to use for each dataset.

    Returns:
    - List of two DataFrames: one for each base model.
    """
    comparison_data_sec = []
    comparison_data_nsec = []

    for task_df, task_name, models, base_model in zip(task_dfs, task_names, model_lists, base_models):
        task_data = {'Task': task_name}
        
        # Check if the base model exists in the dataframe
        if base_model in task_df['Model'].values:
            base_mean = task_df[task_df['Model'] == base_model]['Mean'].values[0]
        else:
            print(f"Base model {base_model} not found in {task_name}")
            continue  # Skip this task if base model not found

        best_model = None
        best_mean = -float('inf')

        # Find the best performing model
        for model in models:
            if model in task_df['Model'].values:
                model_mean = task_df[task_df['Model'] == model]['Mean'].values[0]
                if model_mean > best_mean:
                    best_mean = model_mean
                    best_model = model
            else:
                print(f"Model {model} not found in {task_name}")

        # Populate comparison data, with formatting
        for model in models:
            if model in task_df['Model'].values:
                model_mean = task_df[task_df['Model'] == model]['Mean'].values[0]
                is_significant = task_df[task_df['Model'] == model]['Significant'].values[0] == 'No'

                if model == best_model:
                    # Highlight best performing model (bold + underline)
                    task_data[model] = f"**__{model_mean:.2f}__**"
                elif not is_significant:
                    # Highlight models not statistically significant from SEC (bold)
                    task_data[model] = f"**{model_mean:.2f}**"
                else:
                    # Normal formatting for significant models
                    task_data[model] = f"{model_mean:.2f}"
            else:
                task_data[model] = 'N/A'  # If model not found, mark as N/A
        
        # Append to the appropriate comparison list
        if base_model == "SEC":
            comparison_data_sec.append(task_data)
        else:
            comparison_data_nsec.append(task_data)
    
    # Create DataFrame from comparison data for each base model
    comparison_df_sec = pd.DataFrame(comparison_data_sec)
    comparison_df_nsec = pd.DataFrame(comparison_data_nsec)
    
    return comparison_df_sec, comparison_df_nsec


In [14]:
file_path = '../data/'

# Load the CSV files
memory_sec_df = pd.read_csv(file_path+'SECforgetting_statistical_test_results_new.csv')
memory_nsec_df = pd.read_csv(file_path+'NSECforgetting_statistical_test_results_new.csv')
# Use the function for SEC and NSEC as base models
task_dfs = [memory_sec_df, memory_nsec_df]
task_names = ['SEC forgetting', 'NSEC forgetting']
model_lists = [['SEC', 'SEC-fifo', 'SEC-rwd'], ['NSEC', 'NSEC-fifo', 'NSEC-rwd']]
base_models = ['SEC', 'NSEC']


#print(memory_sec_df['Model'].unique())
#print(memory_nsec_df['Model'].unique())


# Generate the two tables
comparison_df_sec, comparison_df_nsec = generate_two_comparison_tables(
    task_dfs=task_dfs,
    task_names=task_names,
    model_lists=model_lists,
    base_models=base_models
)

In [19]:
# Call the function to display and save the dataframe
display_and_save_dataframe_v3(comparison_df_sec, filename="SECforgetting_studies_performance_comparison_new.csv")



**Model Performance Comparison**



C:\Users\zerat\AppData\Local\Temp\ipykernel_9924\3706623491.py:12: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(results_df.style.hide_index().set_table_styles(


Task,SEC,SEC-fifo,SEC-rwd
SEC forgetting,**11604.88**,**__12401.42__**,12191.91


In [21]:
display_and_save_dataframe_v3(comparison_df_nsec, filename="NSECforgetting_studies_performance_comparison_new.csv")


**Model Performance Comparison**



C:\Users\zerat\AppData\Local\Temp\ipykernel_9924\3706623491.py:12: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(results_df.style.hide_index().set_table_styles(


Task,NSEC,NSEC-fifo,NSEC-rwd
NSEC forgetting,**5024.53**,5173.49,**__6669.10__**


## Integrated table exp2

In [230]:
file_path = '../data/'

# Load the CSV files
memory_sec_df = pd.read_csv(file_path+'Memory Constraints SEC_statistical_test_results.csv')
memory_nsec_df = pd.read_csv(file_path+'Memory Constraints NSEC_statistical_test_results.csv')
# Use the function for SEC and NSEC as base models
task_dfs = [memory_sec_df, memory_nsec_df]
task_names = ['SEC memory constraints', 'NSEC memory constraints']
model_lists = [['SEC_ltm1000' 'SEC_ltm125' 'SEC_ltm250' 'SEC_ltm500'], ['NSEC_ltm1000' 'NSEC_ltm125' 'NSEC_ltm250' 'NSEC_ltm500']]
base_models = ['SEC_ltm1000', 'NSEC_ltm1000']

print(memory_sec_df['Model'].unique())
print(memory_nsec_df['Model'].unique())

# Generate the two tables
comparison_df_sec, comparison_df_nsec = generate_two_comparison_tables(
    task_dfs=task_dfs,
    task_names=task_names,
    model_lists=model_lists,
    base_models=base_models
)

['SEC_ltm1000' 'SEC_ltm125' 'SEC_ltm250' 'SEC_ltm500']
['NSEC_ltm1000' 'NSEC_ltm125' 'NSEC_ltm250' 'NSEC_ltm500']
Model SEC_ltm1000SEC_ltm125SEC_ltm250SEC_ltm500 not found in SEC memory constraints
Model NSEC_ltm1000NSEC_ltm125NSEC_ltm250NSEC_ltm500 not found in NSEC memory constraints


In [231]:
# Call the function to display and save the dataframe
display_and_save_dataframe_v3(comparison_df_sec, filename="SECmemory_studies_performance_comparison.csv")


**Model Performance Comparison**



C:\Users\zerat\AppData\Local\Temp\ipykernel_18988\1298627467.py:14: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(results_df.style.hide_index().set_table_styles(


In [ ]:
# Call the function to display and save the dataframe
display_and_save_dataframe_v3(comparison_df_sec, filename="NSECmemory_studies_performance_comparison.csv")

In [236]:
# Load the uploaded CSV files for memory constraints analysis
memory_sec_df = pd.read_csv(file_path+'Memory Constraints SEC_statistical_test_results.csv')
memory_nsec_df = pd.read_csv(file_path+'Memory Constraints NSEC_statistical_test_results.csv')

# Define task names, model lists, and base models for SEC and NSEC
task_dfs = [memory_sec_df, memory_nsec_df]
task_names = ['SEC memory constraints', 'NSEC memory constraints']
model_lists = [['SEC_ltm1000', 'SEC_ltm125', 'SEC_ltm250', 'SEC_ltm500'],
               ['NSEC_ltm1000', 'NSEC_ltm125', 'NSEC_ltm250', 'NSEC_ltm500']]
base_models = ['SEC_ltm1000', 'NSEC_ltm1000']

# Define the function to generate and save/display the comparison tables
from IPython.display import display

def generate_and_display_memory_tables(task_dfs, task_names, model_lists, base_models):
    """
    Generate two comparison tables with different base models for each dataset,
    display them, and save them as CSV files.

    Parameters:
    - task_dfs: List of DataFrames corresponding to the tasks.
    - task_names: List of task names.
    - model_lists: List of model names for each task.
    - base_models: List of base models to use for each dataset.
    """
    comparison_data_sec = []
    comparison_data_nsec = []

    for task_df, task_name, models, base_model in zip(task_dfs, task_names, model_lists, base_models):
        task_data = {'Task': task_name}
        
        # Check if the base model exists in the dataframe
        if base_model in task_df['Model'].values:
            base_mean = task_df[task_df['Model'] == base_model]['Mean'].values[0]
        else:
            print(f"Base model {base_model} not found in {task_name}")
            continue  # Skip this task if base model not found

        best_model = None
        best_mean = -float('inf')

        # Find the best performing model
        for model in models:
            if model in task_df['Model'].values:
                model_mean = task_df[task_df['Model'] == model]['Mean'].values[0]
                if model_mean > best_mean:
                    best_mean = model_mean
                    best_model = model
            else:
                print(f"Model {model} not found in {task_name}")

        # Populate comparison data, with formatting
        for model in models:
            if model in task_df['Model'].values:
                model_mean = task_df[task_df['Model'] == model]['Mean'].values[0]
                is_significant = task_df[task_df['Model'] == model]['Significant'].values[0] == 'No'

                if model == best_model:
                    # Highlight best performing model (bold + underline)
                    task_data[model] = f"**__{model_mean:.2f}__**"
                elif not is_significant:
                    # Highlight models not statistically significant from base model (bold)
                    task_data[model] = f"**{model_mean:.2f}**"
                else:
                    # Normal formatting for significant models
                    task_data[model] = f"{model_mean:.2f}"
            else:
                task_data[model] = 'N/A'  # If model not found, mark as N/A
        
        # Append to the appropriate comparison list
        if base_model == "SEC_ltm1000":
            comparison_data_sec.append(task_data)
        else:
            comparison_data_nsec.append(task_data)
    
    # Create DataFrames from comparison data for each base model
    comparison_df_sec = pd.DataFrame(comparison_data_sec)
    comparison_df_nsec = pd.DataFrame(comparison_data_nsec)

    # Display and save each DataFrame
    def display_and_save_table(df, table_name, filename):
        print(f"\n**{table_name}**\n")
        display(df.style.set_caption(table_name).hide_index().set_table_styles(
            [
                {'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center')]},
                {'selector': 'td', 'props': [('font-size', '12pt'), ('text-align', 'center')]}
            ]
        ).set_properties(**{'text-align': 'center'}))
        
        df.to_csv(filename, index=False)
        print(f"Table saved as {filename}")

    # Display and save SEC memory constraints table
    display_and_save_table(comparison_df_sec, "SEC Memory Constraints Comparison", "SEC_memory_constraints_comparison.csv")

    # Display and save NSEC memory constraints table
    display_and_save_table(comparison_df_nsec, "NSEC Memory Constraints Comparison", "NSEC_memory_constraints_comparison.csv")

# Run the function with the loaded data
generate_and_display_memory_tables(task_dfs, task_names, model_lists, base_models)



**SEC Memory Constraints Comparison**



C:\Users\zerat\AppData\Local\Temp\ipykernel_18988\1377185438.py:83: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  display(df.style.set_caption(table_name).hide_index().set_table_styles(


Task,SEC_ltm1000,SEC_ltm125,SEC_ltm250,SEC_ltm500
SEC memory constraints,**__12174.31__**,**9241.33**,**10351.39**,12031.59


Table saved as SEC_memory_constraints_comparison.csv

**NSEC Memory Constraints Comparison**



Task,NSEC_ltm1000,NSEC_ltm125,NSEC_ltm250,NSEC_ltm500
NSEC memory constraints,**__7424.22__**,**3817.13**,**5186.52**,6181.66


Table saved as NSEC_memory_constraints_comparison.csv
